In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
#path to pickle of all our datasets from the previous assignment. if not present run code in assignment1.ipynb
pickle_file = '../dataset/arbimonTest1.pickle'

#open and load pickle file. extract each one of the datasets 
with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (76, 100, 100) (76,)
Validation set (47, 100, 100) (47,)
Test set (47, 100, 100) (47,)


In [27]:
image_size = 100
num_labels = 9

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]. 
    # i.e. np.arange creates array from 0 to 9, compares each index in that list to each index in the 1D labels array
    # 0 as [True, False, False,...], 1 as [False, True, False,...] then cast all as float
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

#reshape and reformat datasets and labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (76, 10000) (2,)
Validation set (47, 10000) (2,)
Test set (47, 10000) (2,)


In [28]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 76

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    # creates constant tensor to hold values of datasets
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    # draw random values from truncated normal distribution with mean 0.0, stddev 1.0.
    #outputs float32 tensor of shape (784, 10)
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    # 1D array of zeroes. shape(num_labels, )
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    # run operation WX + b where X is input from your dataset
    logits = tf.matmul(tf_train_dataset, weights) + biases
    print(logits)
    #tf.reshape(tf_train_labels, [76,2])
    loss = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Tensor("add:0", shape=(76, 2), dtype=float32)


ValueError: Dimensions must be equal, but are 76 and 2 for 'SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' (op: 'SparseSoftmaxCrossEntropyWithLogits') with input shapes: [76,2], [2].

In [7]:
num_steps = 801

#calculate accuracy from training
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.295246
Training accuracy: 6.0%
Validation accuracy: 8.6%
Loss at step 100: 0.006445
Training accuracy: 100.0%
Validation accuracy: 34.3%
Loss at step 200: 0.003152
Training accuracy: 100.0%
Validation accuracy: 34.3%
Loss at step 300: 0.002101
Training accuracy: 100.0%
Validation accuracy: 34.3%
Loss at step 400: 0.001580
Training accuracy: 100.0%
Validation accuracy: 34.3%
Loss at step 500: 0.001268
Training accuracy: 100.0%
Validation accuracy: 31.4%
Loss at step 600: 0.001060
Training accuracy: 100.0%
Validation accuracy: 31.4%
Loss at step 700: 0.000911
Training accuracy: 100.0%
Validation accuracy: 31.4%
Loss at step 800: 0.000799
Training accuracy: 100.0%
Validation accuracy: 31.4%
Test accuracy: 28.6%


In [20]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [21]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized


ValueError: Cannot feed value of shape (70, 10000) for Tensor 'Placeholder:0', which has shape '(128, 10000)'

In [ ]:
# From notes from course:
# a simple 2 layer neural network consists of a set of weights and biases applied to X (WX + b) 
# whose output is passed through H number of ReLUs. The output of passing through RELUs is passed 
# to the next set of weights and biases applied to this intermediate output. The output of the RELUs 
# is not visible outside of the network so these layers of RELUs are called hidden layers. 
# Whole process so far (WX + b) -> z -> RELUs(z) -> I -> WI + b -> y -> Softmax(y) -> scores (0-1)
# where z is the result of a matrix multiply, I is a RELUs intermediatary output, and y are logits

#set up our neural network with 1024 hidden nodes

batch_size = 128
h_nodes = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Constants
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, h_nodes]))
    b1 = tf.Variable(tf.zeros([h_nodes]))
    W2 = tf.Variable(tf.truncated_normal([h_nodes, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))    

    # Training computation.
    L1 = tf.matmul(tf_train_dataset, W1) + b1
    h_layer = tf.nn.relu(L1)
    logits = tf.matmul(h_layer, W2) + b2 
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    L1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1)
    valid_prediction = tf.nn.softmax(tf.matmul(L1_valid, W2) + b2)
    L1_test = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + b1)
    test_prediction = tf.nn.softmax(tf.matmul(L1_test, W2) + b2)

In [ ]:
num_steps = 10000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))